In [89]:
import pandas as pd
from langdetect import detect
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import CountVectorizer
import math
import matplotlib.pyplot

In [90]:
lem = WordNetLemmatizer() #create lemmatizer

In [91]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kseniapavlina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [92]:
dictionary = pd.read_csv('../data/processedDict.csv')
dictionary

,word,cat_lev1,cat_lev2,cat_lev3,ndf
0,information,business,professional_conduct_host,advice,0.008904
1,recommendations,business,professional_conduct_host,advice,0.008375
2,tips,business,professional_conduct_host,advice,0.008399
3,advice,business,professional_conduct_host,advice,0.007568
4,suggestions,business,professional_conduct_host,advice,0.007580
...,...,...,...,...,...
347,talking,social,social_interaction,talking,0.001961
348,chatting,social,social_interaction,talking,0.001408
349,moments,social,social_interaction,talking,0.001155
350,chats,social,social_interaction,talking,0.000553


In [93]:
dictionary['word'] = dictionary['word'].apply(lambda x: lem.lemmatize(x, pos='n'))

In [94]:
filepath = '../data/en_reviews/belize_en.csv'
reviews = pd.read_csv(filepath)

# reviews = reviews['date']
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments,langs,dict_proportion
0,36666,145174,27/11/2010,278365,Rb,Monique and Maurice has put Caye Caulker at th...,en,0.039581
1,36666,167715,08/01/2011,258142,Angellini,"We had the best time, food and conversations w...",en,0.031722
2,36666,172375,19/01/2011,268097,Michael,"Great, great place to stay on Caye Caulker. Ve...",en,0.037152
3,36666,179895,08/02/2011,340713,Melinda,We stayed with Monique and Maurice for a coupl...,en,0.046602
4,36666,181056,12/02/2011,117024,Gail,Monique's guest house was great for us. It was...,en,0.055441
...,...,...,...,...,...,...,...,...
37705,39127463,557741654,01/11/2019,22844580,Cheryl,"Very clean, new place. There are 2 units, we ...",en,0.064935
37706,39183755,550911001,19/10/2019,41897670,Alice,Brandon’s room at Caye caulker is basic but co...,en,0.048851
37707,39183755,562960822,11/11/2019,23535832,Clara,Great spot! Loved having AC and a pool! \nBage...,en,0.025641
37708,39944342,565690422,17/11/2019,25461269,Moira,Great place!,en,0.090909


In [95]:
table = str.maketrans('', '', string.punctuation) #mapping to strip punctuation in review

In [96]:
#strip punct of each review -> lemmatise -> output is list of words so join into sentences
reviews['comments'] = reviews.comments.apply(lambda review: ' '.join(map(str, [lem.lemmatize(word.translate(table), pos='n') for word in review.lower().split()])))
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments,langs,dict_proportion
0,36666,145174,27/11/2010,278365,Rb,monique and maurice ha put caye caulker at the...,en,0.039581
1,36666,167715,08/01/2011,258142,Angellini,we had the best time food and conversation whi...,en,0.031722
2,36666,172375,19/01/2011,268097,Michael,great great place to stay on caye caulker very...,en,0.037152
3,36666,179895,08/02/2011,340713,Melinda,we stayed with monique and maurice for a coupl...,en,0.046602
4,36666,181056,12/02/2011,117024,Gail,moniques guest house wa great for u it wa exac...,en,0.055441
...,...,...,...,...,...,...,...,...
37705,39127463,557741654,01/11/2019,22844580,Cheryl,very clean new place there are 2 unit we were ...,en,0.064935
37706,39183755,550911001,19/10/2019,41897670,Alice,brandon’s room at caye caulker is basic but co...,en,0.048851
37707,39183755,562960822,11/11/2019,23535832,Clara,great spot loved having ac and a pool bagel in...,en,0.025641
37708,39944342,565690422,17/11/2019,25461269,Moira,great place,en,0.090909


In [97]:
def logTermFrequency(word, review):
    totalNumberOfWords = review.split()
    totalNumberOfWords = len(totalNumberOfWords)
    numberOfWord = review.count(word)
    if numberOfWord == 0:
        return 0;
    result = numberOfWord * 100/ totalNumberOfWords
    return result   

In [98]:
def adoptionForOneReview(category, review, dictionary):
    adoption = 0
    dictionaryWords = dictionary[(dictionary['cat_lev1'] == category)]
    dictionaryWords = dictionaryWords['word']
    for word in dictionaryWords:
        adoption = adoption + logTermFrequency(word, review)
    return adoption

In [ ]:
reviews['business'] = reviews['comments'].apply(lambda x: adoptionForOneReview('business', x, dictionary))
reviews['social'] = reviews['comments'].apply(lambda x: adoptionForOneReview('social', x, dictionary))
reviews

In [ ]:
k_social = reviews['social'].loc[reviews['social'] > 0]
k_social = k_social.min()
k_social

In [ ]:
k_business = reviews['business'].loc[reviews['business'] > 0]
k_business = k_business.min()
k_business

In [ ]:
reviews['date'] = pd.to_datetime(reviews['date'])

In [ ]:
def adoptionForSetOfReviews(category, setOfReviews, dictionary, startDate, endDate, k):
    adoption = 1
    mask = (setOfReviews['date'] > startDate) & (setOfReviews['date'] < endDate)
    setOfReviews = setOfReviews.loc[mask]
    setOfReviews = setOfReviews[category]
    numOfReviews = len(setOfReviews)
    for review in setOfReviews:
        adoption *= (review + k)**(1 / numOfReviews)
    adoption = adoption - k
    return adoption

In [ ]:
mask = (reviews['date'] > '2014-01-01') & (reviews['date'] < '2015-01-01')
setOfReviews = reviews.loc[mask]
setOfReviews

In [ ]:
for2010b = adoptionForSetOfReviews('business', reviews, dictionary, '2010-01-01', '2011-01-01', k_business)
for2011b = adoptionForSetOfReviews('business', reviews, dictionary, '2011-01-01', '2012-01-01', k_business)
for2012b = adoptionForSetOfReviews('business', reviews, dictionary, '2012-01-01', '2013-01-01', k_business)
for2013b = adoptionForSetOfReviews('business', reviews, dictionary, '2013-01-01', '2014-01-01', k_business)
for2014b = adoptionForSetOfReviews('business', reviews, dictionary, '2014-01-01', '2015-01-01', k_business)
for2015b = adoptionForSetOfReviews('business', reviews, dictionary, '2015-01-01', '2016-01-01', k_business)
for2016b = adoptionForSetOfReviews('business', reviews, dictionary, '2016-01-01', '2017-01-01', k_business)
for2017b = adoptionForSetOfReviews('business', reviews, dictionary, '2017-01-01', '2018-01-01', k_business)
for2018b = adoptionForSetOfReviews('business', reviews, dictionary, '2018-01-01', '2019-01-01', k_business)
for2019b = adoptionForSetOfReviews('business', reviews, dictionary, '2019-01-01', '2020-01-01', k_business)

print("business")
print("2010 = ",for2010b)
print("2011 = ",for2011b)
print("2012 = ",for2012b)
print("2013 = ",for2013b)
print("2014 = ",for2014b)
print("2015 = ",for2015b)
print("2016 = ",for2016b)
print("2017 = ",for2017b)
print("2018 = ",for2018b)
print("2019 = ",for2019b)




In [ ]:
for2010 = adoptionForSetOfReviews('social', reviews, dictionary, '2010-01-01', '2011-01-01', k_social)
for2011 = adoptionForSetOfReviews('social', reviews, dictionary, '2011-01-01', '2012-01-01', k_social)
for2012 = adoptionForSetOfReviews('social', reviews, dictionary, '2012-01-01', '2013-01-01', k_social)
for2013 = adoptionForSetOfReviews('social', reviews, dictionary, '2013-01-01', '2014-01-01', k_social)
for2014 = adoptionForSetOfReviews('social', reviews, dictionary, '2014-01-01', '2015-01-01', k_social)
for2015 = adoptionForSetOfReviews('social', reviews, dictionary, '2015-01-01', '2016-01-01', k_social)
for2016 = adoptionForSetOfReviews('social', reviews, dictionary, '2016-01-01', '2017-01-01', k_social)
for2017 = adoptionForSetOfReviews('social', reviews, dictionary, '2017-01-01', '2018-01-01', k_social)
for2018 = adoptionForSetOfReviews('social', reviews, dictionary, '2018-01-01', '2019-01-01', k_social)
for2019 = adoptionForSetOfReviews('social', reviews, dictionary, '2019-01-01', '2020-01-01', k_social)

print("social")
print("2010 = ",for2010)
print("2011 = ",for2011)
print("2012 = ",for2012)
print("2013 = ",for2013)
print("2014 = ",for2014)
print("2015 = ",for2015)
print("2016 = ",for2016)
print("2017 = ",for2017)
print("2018 = ",for2018)
print("2019 = ",for2019)

In [ ]:
x_axis = [for2010b, for2011b, for2012b, for2013b, for2014b, for2015b, for2016b, for2017b, for2018b, for2019b]
df = pd.DataFrame(x_axis, index = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019'], columns = ['business adoption'])
df
df.plot.bar()

In [ ]:
x_axis = [for2010, for2011, for2012, for2013, for2014, for2015, for2016, for2017, for2018, for2019]
df = pd.DataFrame(x_axis, index = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019'], columns = ['social adoption'])
df
df.plot.bar()